In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 定义LeNet-5网络结构
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # 第一层卷积: 1个输入通道, 6个输出通道, 5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        # 第二层卷积: 6个输入通道, 16个输出通道, 5x5卷积核
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        # 全连接层
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # 激活函数和池化层
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        # 第一层: 卷积 -> ReLU -> 池化
        x = self.pool(self.relu(self.conv1(x)))
        # 第二层: 卷积 -> ReLU -> 池化
        x = self.pool(self.relu(self.conv2(x)))
        # 展平
        x = x.view(-1, 16 * 5 * 5)
        # 全连接层
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# 训练函数
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # 前向传播
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        # 统计
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
    accuracy = 100. * correct / total
    avg_loss = train_loss / len(train_loader)
    print(f'Train set: Average loss: {avg_loss:.4f}, Accuracy: {correct}/{total} ({accuracy:.2f}%)')
    return avg_loss, accuracy


In [5]:
# 测试函数
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
    
    test_loss /= len(test_loader)
    accuracy = 100. * correct / total
    
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{total} ({accuracy:.2f}%)\n')
    return test_loss, accuracy

In [ ]:
# 可视化训练结果
def plot_results(train_losses, train_accs, test_losses, test_accs):
    epochs = range(1, len(train_losses) + 1)
    
    plt.figure(figsize=(12, 4))
    
    # 损失曲线
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Train Loss')
    plt.plot(epochs, test_losses, 'r-', label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.grid(True)
    
    # 准确率曲线
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, 'b-', label='Train Accuracy')
    plt.plot(epochs, test_accs, 'r-', label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('outputs/training_results.png', dpi=300, bbox_inches='tight')
    print("训练结果图已保存到 training_results.png")

In [ ]:
# 显示一些预测结果
def visualize_predictions(model, device, test_loader, num_images=10):
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images[:num_images].to(device), labels[:num_images]
    
    with torch.no_grad():
        outputs = model(images)
        _, predictions = outputs.max(1)
    
    images = images.cpu()
    predictions = predictions.cpu()
    
    plt.figure(figsize=(15, 3))
    for i in range(num_images):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        plt.title(f'True: {labels[i]}\nPred: {predictions[i]}',
                  color='green' if labels[i] == predictions[i] else 'red')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('outputs/predictions.png', dpi=300, bbox_inches='tight')
    print("预测结果图已保存到 predictions.png")

In [8]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    
    # 超参数设置
    batch_size = 64
    learning_rate = 0.001
    num_epochs = 10
    
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'使用设备: {device}')
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST数据集的均值和标准差
    ])
    
    # 加载数据集
    print("正在下载和加载MNIST数据集...")
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print(f"训练集大小: {len(train_dataset)}")
    print(f"测试集大小: {len(test_dataset)}")
    
    # 创建模型
    model = LeNet5().to(device)
    print("\nLeNet-5 网络结构:")
    print(model)
    
    # 计算模型参数量
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n总参数量: {total_params:,}")
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # 训练模型
    train_losses, train_accs = [], []
    test_losses, test_accs = [], []
    
    print("\n开始训练...")
    for epoch in range(1, num_epochs + 1):
        print(f"\n{'='*50}")
        train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
        test_loss, test_acc = test(model, device, test_loader, criterion)
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
    
    # 保存模型
    torch.save(model.state_dict(), '/mnt/user-data/outputs/lenet5_mnist.pth')
    print("模型已保存到 lenet5_mnist.pth")
    
    # 可视化结果
    plot_results(train_losses, train_accs, test_losses, test_accs)
    visualize_predictions(model, device, test_loader)
    
    print("\n训练完成!")
    print(f"最终测试准确率: {test_accs[-1]:.2f}%")


if __name__ == '__main__':
    main()


使用设备: cuda
正在下载和加载MNIST数据集...


100%|██████████| 9.91M/9.91M [03:05<00:00, 53.5kB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 138kB/s]
100%|██████████| 1.65M/1.65M [00:20<00:00, 79.0kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.2MB/s]


训练集大小: 60000
测试集大小: 10000

LeNet-5 网络结构:
LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

总参数量: 61,706

开始训练...

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304433
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.446174
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.244137
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.136241
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.162051
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.206196
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.077167
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.342371
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.076813
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.048798
Train set: 

RuntimeError: Parent directory /mnt/user-data/outputs does not exist.